### Install required external libraries 

In [0]:
%pip install transformers==4.30.2 "unstructured[pdf,docx]==0.10.30" langchain==0.1.5 llama-index==0.9.3 databricks-vectorsearch==0.22 pydantic==1.10.9 mlflow==2.10.1
dbutils.library.restartPython()

  Obtaining dependency information for transformers==4.30.2 from https://files.pythonhosted.org/packages/5b/0b/e45d26ccd28568013523e04f325432ea88a442b4e3020b757cf4361f0120/transformers-4.30.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/113.6 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 3.2 MB/s eta 0:00:00
  Obtaining dependency information for unstructured[docx,pdf]==0.10.30 from https://files.pythonhosted.org/packages/e3/95/28c08bbdd98a9bdbb7f52c07c0e81090ea5f89af5b6cac7c7c6d86ce2a5e/unstructured-0.10.30-py3-none-any.whl.metadata
  Obtaining dependency information for langchain==0.1.5 from https://files.pythonhosted.org/packages/c1/c3/0e59a0c24e0c61b52271445df55302ab2f3dd8489a365721c7ef7ecaba24/langchain-0.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index==0.9.3 from https://files.pythonhosted.org/packages/e3/fa/94e09b6b35efa90451e060cc1ac1a32c91a13d65fee6371bf7c401aa971a/llama_index-0.9.3-

### Init our resources and catalog

In [0]:
%run ./00-init $reset_all_data=false

###pdf files are available in Volume (or DBFS)

In [0]:
# List our raw PDF docs
volume_folder =  f"/Volumes/{catalog}/{db}/documents_store/pdf_documents"   

display(dbutils.fs.ls(volume_folder))

path,name,size,modificationTime
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Financial Report - December 2023.pdf,Contoso Quarterly Financial Report - December 2023.pdf,137216,1753967249000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Financial Report - March 2024.pdf,Contoso Quarterly Financial Report - March 2024.pdf,111983,1753967249000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Market SentimentReport - December 2023.pdf,Contoso Quarterly Market SentimentReport - December 2023.pdf,207724,1753967250000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso Quarterly Market SentimentReport - March 2024.pdf,Contoso Quarterly Market SentimentReport - March 2024.pdf,203184,1753967250000
dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso_improve_customer_churn.pdf,Contoso_improve_customer_churn.pdf,634144,1753967249000


### Transform pdf as text

In [0]:
from unstructured.partition.auto import partition
import re

def extract_doc_text(x : bytes) -> str:
  # Read files and extract the values with unstructured
  sections = partition(file=io.BytesIO(x))
  def clean_section(txt):
    txt = re.sub(r'\n', '', txt)
    return re.sub(r' ?\.', '.', txt)
  # Default split is by section of document, concatenate them all together because we want to split by sentence instead.
  return "\n".join([clean_section(s.text) for s in sections]) 

2025-07-31 17:23:10.695789: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-31 17:23:10.827926: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-31 17:23:11.409588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-31 17:23:16.951225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [0]:
%sql
--Note that we need to enable Change Data Feed on the table to create the index
CREATE TABLE IF NOT EXISTS litware_unity_catalog.rag.documents_embedding (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  url STRING,
  content STRING,
  embedding STRING
) TBLPROPERTIES (delta.enableChangeDataFeed = true); 

#### Create the final documents_embedding table containing chunks and embeddings

In [0]:
from pyspark.sql import SparkSession, functions as F
import io
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index import Document, set_global_tokenizer
from transformers import AutoTokenizer

# Reduce the arrow batch size as our PDF can be big in memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)

# Define the function to read data as chunks
def read_as_chunk(text):
    # set llama2 as tokenizer to match our model size (will stay below BGE 1024 limit)
    set_global_tokenizer(
        AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
    )
    # Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
    txt = extract_doc_text(text)
    nodes = splitter.get_nodes_from_documents([Document(text=txt)])
    return [n.text for n in nodes]

# Register the UDF
read_as_chunk_udf = F.udf(read_as_chunk, F.ArrayType(F.StringType()))

# Read data from the 'pdf_raw' table
df = spark.read.table(f"{catalog}.{db}.documents_raw")

# Use withColumn and the UDF to add a new column
df = df.withColumn("text", F.explode(read_as_chunk_udf(df.content)))

# Show the modified DataFrame
df.show(2)

+--------------------+-------------------+------+--------------------+--------------------+
|                path|   modificationTime|length|             content|                text|
+--------------------+-------------------+------+--------------------+--------------------+
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|Quarterly Market ...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|In summary, Conto...|
+--------------------+-------------------+------+--------------------+--------------------+
only showing top 2 rows



### Computing the chunk embeddings and saving them to our Delta Table


In [0]:
import mlflow.deployments
from pyspark.sql import functions as F
from pyspark.sql.functions import *

def get_embedding(text: str) -> list[float]:
    deploy_client = mlflow.deployments.get_deploy_client("databricks")  # Assuming MLflow is configured

    try:
        response = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": [text]})
        return response.data[0]['embedding']  # Extract embedding for the single input
    except Exception as e:
        return None  # Handle potential errors

# Read data from the 'pdf_raw' table
#df = spark.read.table(f"{catalog}.{db}.fabcon_pdf_raw")

get_embedding_udf = F.udf(get_embedding, F.StringType())  # Adjusted return type for single string
df = df.withColumn("embedding", get_embedding_udf(F.col("text")))  # Apply UDF to text column

df.show(2)

+--------------------+-------------------+------+--------------------+--------------------+--------------------+
|                path|   modificationTime|length|             content|                text|           embedding|
+--------------------+-------------------+------+--------------------+--------------------+--------------------+
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|Quarterly Market ...|[0.00300407409667...|
|dbfs:/Volumes/lit...|2025-07-31 13:07:29|634144|[25 50 44 46 2D 3...|In summary, Conto...|[0.01162719726562...|
+--------------------+-------------------+------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [0]:
from pyspark.sql.functions import concat

df.selectExpr('path as url', 'text as content', 'embedding') \
  .write \
  .mode("append") \
  .format("delta") \
  .saveAsTable(f"{catalog}.{db}.documents_embedding")

In [0]:
%sql
SELECT * FROM litware_unity_catalog.rag.documents_embedding WHERE url like '%.pdf' limit 6

id url content embedding 1 dbfs:/Volumes/litware_unity_catalog/rag/documents_store/pdf_documents/Contoso_improve_customer_churn.pdf Quarterly Market Sentiment Report December 2023
Summary:
Contoso, a retail company headquartered in the US, is currently facing a challenging market sentiment scenario with a score of 40%, which falls below industry averages. This decline in market sentiment, especially among key segments such as Millennials, raises concerns for investors regarding the company's growth, profitability, and competitive positioning. The downward trend in sentiment over the past few quarters indicates a worsening perception and potential loss of investor confidence, leading to a negative brand perception among customers.
© 2023 Contoso Confidential
One of the primary factors contributing to Contoso's poor market sentiment is its lack of focus on Environmental, Social, and Governance (ESG) initiatives. Customers perceive Contoso as failing to address sustainability concerns, such as excessive packaging and unsustainable sourcing practices. As a result, eco-conscious consumers are shifting their preferences towards more environmentally friendly competitors. Additionally, environmental scandals and controversies have tarnished Contoso's reputation, leading to boycotts and further deteriorating customer satisfaction and market sentiment. Negative customer experiences and complaints shared on social media platforms are amplifying these concerns and exacerbating the decline in market sentiment.
The customer reviews data provides further insight into Contoso's poor market sentiment. A significant number of customers express extreme dissatisfaction with the products and services provided by Contoso. Issues such as poor product quality, unresponsive customer service, and unresolved problems are prevalent themes across these reviews. Customers express regret over their purchases and indicate that they would not recommend Contoso's products to others, further contributing to the negative market sentiment surrounding the company.
In summary, Contoso's lack of focus on ESG initiatives, coupled with widespread customer dissatisfaction highlighted in the reviews, is driving the company's poor market sentiment. [0.0030040740966796875, 0.02423095703125, -0.0280914306640625, 0.0293121337890625, -0.01248931884765625, -0.039306640625, 0.0233001708984375, 0.0343017578125, 0.038360595703125, 0.04656982421875, -0.01232147216796875, 0.01102447509765625, 0.0047454833984375, -0.0205535888671875, -0.01666259765625, -0.03515625, 0.0158843994140625, 0.009979248046875, -0.0035037994384765625, 0.00826263427734375, 0.002666473388671875, 0.01206207275390625, -0.0204620361328125, -0.0017309188842773438, -0.0380859375, 0.033935546875, 0.004817962646484375, -0.00860595703125, 0.07293701171875, 0.022918701171875, -0.0022792816162109375, -0.0161895751953125, 0.0240478515625, -0.039276123046875, 0.007289886474609375, -9.388923645019531E-4, 0.020111083984375, -0.0120086669921875, -0.0239105224609375, -0.044219970703125, -0.004772186279296875, -0.0165863037109375, 0.06732177734375, -0.022796630859375, -0.01314544677734375, -0.0261077880859375, -0.016998291015625, -0.046905517578125, -0.0221405029296875, -0.0201873779296875, 0.024017333984375, -0.0039825439453125, 0.0155029296875, -0.006805419921875, 0.03424072265625, -0.0240020751953125, -0.01348876953125, 0.0139923095703125, -0.020843505859375, 0.0445556640625, 0.067138671875, -0.00968170166015625, 0.04119873046875, -0.040313720703125, 0.01116943359375, 0.00653839111328125, -0.0098724365234375, -0.0069122314453125, -0.009796142578125, 4.627704620361328E-4, -0.0159912109375, 0.004825592041015625, -0.039031982421875, -0.038421630859375, -0.00936126708984375, -0.0156402587890625, 0.005069732666015625, -0.020782470703125, -0.0301513671875, -0.0233154296875, 0.00200653076171875, -0.01922607421875, 0.0472412109375, 0.01497650146484375, -0.05126953125, 0.0357666015625, 0.0279388427734375, -0.00165557861328125, -0.